In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.tsa as tsa
import statsmodels.formula.api as smf

In [ ]:
def time_weighted_average(rows, statistic):
    stat = rows[statistic]
    if (rows.shape[0] == 5):
        return .297*stat[4] + .297*.8*stat[3] + .297*(.8^2)*stat[2] + .297*(.8^3)*stat[1] + .297*(.8^4)*stat[0]
    else if (rows.shape[0] == 10):
        return .224*stat[9] + .224*.8*stat[8] + .224*(.8^2)*stat[7] + .224*(.8^3)*stat[6] + .224*(.8^4)*stat[5] + .224*(.8^5)*stat[4] + .224*(.8^6)*stat[3] + .224*(.8^7)*stat[2] + .297*(.8^4)*stat[0]

In [1]:
player_box_scores = pd.read_csv('SAGB-master/all_games_new.csv')

input_statistics = ["made_field_goals", "attempted_field_goals", "made_three_point_field_goals", "attempted_three_point_field_goals", "attempted_free_throws", "made_free_throws", "offensive_rebounds", "defensive_rebounds", "assists", "blocks", "turnovers", "steals"]
for (box_score in player_box_scores):
    player_name = box_score["name"]
    game_date = box_score["Date"]
    last_5_rows = get_stats(player_name, game_date, 5)
    weighted_stats = []
    for (stat in input_statistics):
        weighted_stats.append(time_weighted_average(last_5_rows, stat))
    line_with_averages = 

SyntaxError: invalid syntax (<ipython-input-1-b75340425eaf>, line 3)